In [1]:
import sympy as sp
import numpy as np
from sympy.physics.vector import dynamicsymbols
from sympy.physics.vector import time_derivative
from sympy.physics.vector import ReferenceFrame
N = ReferenceFrame('N')
import pylab as pl
import control
from EoM import *
from sympy.physics.mechanics import *
from numpy.linalg import matrix_rank, eig
import math
import intelligent_robotics as ir
import numpy as np
from scipy.integrate import odeint
import matplotlib.pyplot as plt
import WIP_utils as utils
import modern_robotics as mr
import time as t
import Cal_joint as cj
import state_equation_pitch as sep
from dmpc import *


In [38]:
I_bx, I_fx, I_fy, m_all, w, l, l_f, f_d, T_gb, g, r, L = sp.symbols('I_bx, I_fx, I_fy, m_all, w, l, l_f, f_d, T_gb, g, r, L')
K_t, K_e, R_m, J_m, V, n, f, b_m = sp.symbols('K_t, K_e, R_m, J_m, V, n, f, b_m')
theta_R, theta_gb = dynamicsymbols('theta_R, theta_gb')

theta_Rd = theta_R.diff()
theta_Rdd = theta_Rd.diff()
theta_gbd = theta_gb.diff()
theta_gbdd = theta_gbd.diff()

q = sp.Matrix([[theta_R], [theta_gb]])
qd = q.diff()
qdd = qd.diff()

u = sp.Matrix([T_gb])

L = I_fy * w / 2

T_gb = -n**2 * J_m * theta_gbdd - n**2 * (K_e*K_t/R_m + b_m)*theta_gbd + (n * K_t * V)/R_m

tau = sp.Matrix([[I_bx*theta_Rdd - m_all*g*l*sp.sin(theta_R) + 2*L*theta_gbd*sp.cos(theta_gb)],
                [I_fx*theta_gbdd -  L*theta_Rd*sp.cos(theta_gb) - T_gb]])
eq_point = {sp.sin(theta_gb):theta_gb, sp.cos(theta_gb):1,sp.sin(theta_R):theta_R, sp.cos(theta_R):1,theta_gbd**2:0 ,theta_Rd**2:0}

tau_eq = sp.simplify(tau.subs(eq_point))


In [39]:
tau_eq

Matrix([
[                                                                                                       I_bx*Derivative(theta_R(t), (t, 2)) + I_fy*w*Derivative(theta_gb(t), t) - g*l*m_all*theta_R(t)],
[(-K_t*V*n + R_m*(2*I_fx*Derivative(theta_gb(t), (t, 2)) - I_fy*w*Derivative(theta_R(t), t) + 2*J_m*n**2*Derivative(theta_gb(t), (t, 2)))/2 + n**2*(K_e*K_t + R_m*b_m)*Derivative(theta_gb(t), t))/R_m]])

In [40]:
u_v = sp.Matrix([V])

In [41]:
Ml, Cl, Gl, Wl = get_EoM_from_T(tau_eq,qdd,g,u_v)

In [42]:
Mlp_inv = Ml.inv()
qdd_rhs_A = Mlp_inv*(-Cl -Gl)
qdd_rhs_B = Mlp_inv*Wl*u

X = q.col_join(qd)
Xd_A = qd.col_join(qdd_rhs_A)
Xd_B = qd.col_join(qdd_rhs_B)
U = u

A = Xd_A.jacobian(X)
B = Xd_B.jacobian(U)
C = X.jacobian(X)
D = X.jacobian(U)

In [43]:
sp.simplify(A)

Matrix([
[             0, 0,                            1,                                                 0],
[             0, 0,                            0,                                                 1],
[g*l*m_all/I_bx, 0,                            0,                                      -I_fy*w/I_bx],
[             0, 0, I_fy*w/(2*(I_fx + J_m*n**2)), -n**2*(K_e*K_t + R_m*b_m)/(R_m*(I_fx + J_m*n**2))]])

In [28]:
B

Matrix([
[                            0],
[                            0],
[                            0],
[K_t*n/(R_m*(I_fx + J_m*n**2))]])

In [29]:
flywheel_ang_vel = (1500 * 2 * np.pi)/60 
# I_bx:9.448246482
param = {w:flywheel_ang_vel, I_fx:0.006235, I_fy:0.0119107, I_bx:0.458398057,  m_all:15, l: 0.5,  g:9.81, n:6, J_m: 0.08, K_e:0.88, K_t: 0.88, R_m:0.33, b_m:0}#22.031

Mlp = msubs(Ml, param)
Clp = msubs(Cl, param)
Glp = msubs(Gl, param)
Wlp = msubs(Wl, param)

Mlp_inv = Mlp.inv()
qdd_rhs_A = Mlp_inv*(-Clp -Glp)
qdd_rhs_B = Mlp_inv*Wlp*u

X = q.col_join(qd)
Xd_A = qd.col_join(qdd_rhs_A)
Xd_B = qd.col_join(qdd_rhs_B)
U = u

A = Xd_A.jacobian(X)
B = Xd_B.jacobian(U)
C = X.jacobian(X)
D = X.jacobian(U)

In [30]:
A

Matrix([
[               0, 0,                 1,                 0],
[               0, 0,                 0,                 1],
[160.504607025418, 0,                 0, -4.08144919548731],
[               0, 0, 0.324112274460603, -29.2699658898184]])

In [31]:
B

Matrix([
[               0],
[               0],
[               0],
[5.54355414579894]])